<a href="https://colab.research.google.com/github/FabianGonzalezz/recomendacion_amazon/blob/master/eda_toys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=4915edbc6161cbc71b71e3a3b89018f3024af94682a765ed70c9ccfbca5705d6
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
from google.colab import drive

In [4]:
from pyspark.sql import SparkSession

In [6]:
folder = 'https://drive.google.com/drive/u/0/folders/1KT0-qPYJmlr6w2o41sjJlPXCZ3SkHbN0'

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [10]:
# Crea una sesión de Spark
spark = SparkSession.builder.appName("Lectura_de_Archivo").getOrCreate()

# Ruta al archivo JSON dentro de la carpeta compartida
json_file_path = '/content/drive/My Drive/Version 2018/Toys_and_Games.json.gz'


In [14]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, BooleanType

In [23]:
custom_schema = StructType([
    StructField("asin", StringType(), nullable=True),
    StructField("image", StringType(), nullable=True),
    StructField("overall", FloatType(), nullable=True),
    StructField("reviewText", StringType(), nullable=True),
    StructField("reviewTime", StringType(), nullable=True),
    StructField("reviewerID", StringType(), nullable=True),
    StructField("reviewerName", StringType(), nullable=True),
    StructField("style", StringType(), nullable=True),
    StructField("summary", StringType(), nullable=True),
    StructField("unixReviewTime", IntegerType(), nullable=True),
    StructField("verified", BooleanType(), nullable=True),
    StructField("vote", StringType(), nullable=True)
])
df = spark.read.json(json_file_path, schema=custom_schema)

In [24]:
df.show()

+----------+--------------------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|               image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+--------------------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|0020232233|                null|    2.0|When it comes to ...|09 22, 2016|A1IDMI31WEANAF|      Mackenzie Kent|                null|The fact that 50%...|    1474502400|   false|  12|
|0020232233|                null|    1.0|An Open Letter to...|09 18, 2016| A4BCEVVZ4Y3V3|  Jonathan Christian|                null|Another worthless...|    1474156800|   false|  21|
|0020232233|                null|    3.0|Nice art, nice pr...|09 12, 2016|A2EZ9PY1IHHBX0| 

In [18]:
df.count()

8201231

In [26]:
from pyspark.sql.functions import from_unixtime, year

# Convertir unixtime en tiempo y extraer el anio para realizar el filtrado

In [27]:
df = df.withColumn("review_date", from_unixtime(df["unixreviewtime"]))

In [29]:
df = df.withColumn("review_year", year(df["review_date"]))

In [30]:
df = df.drop("review_date", "unixreviewtime", 'reviewTime')

In [31]:
df.show()

+----------+--------------------+-------+--------------------+--------------+--------------------+--------------------+--------------------+--------+----+-----------+
|      asin|               image|overall|          reviewText|    reviewerID|        reviewerName|               style|             summary|verified|vote|review_year|
+----------+--------------------+-------+--------------------+--------------+--------------------+--------------------+--------------------+--------+----+-----------+
|0020232233|                null|    2.0|When it comes to ...|A1IDMI31WEANAF|      Mackenzie Kent|                null|The fact that 50%...|   false|  12|       2016|
|0020232233|                null|    1.0|An Open Letter to...| A4BCEVVZ4Y3V3|  Jonathan Christian|                null|Another worthless...|   false|  21|       2016|
|0020232233|                null|    3.0|Nice art, nice pr...|A2EZ9PY1IHHBX0|     unpreparedtodie|                null|pretty, but also ...|    true|  19|       2016

# Filtrado del df entre 2004 y 2014

In [32]:
# Filtrar las filas para incluir solo los años entre 2004 y 2014
df_filtrado = df.filter((df["review_year"] >= 2004) & (df["review_year"] <= 2014))

In [34]:
df_filtrado.count()

2393317

# Al filtrar entre 2004 y 2014 pasamos de 8M de registros a 2M de regsitros.

# No vemos outliers en la columna overall

In [36]:
df_filtrado.select('overall').distinct().show()

+-------+
|overall|
+-------+
|    5.0|
|    2.0|
|    3.0|
|    1.0|
|    4.0|
+-------+



In [37]:
df_meta = spark.read.json('/content/drive/My Drive/Version 2018/meta_Toys_and_Games.json.gz')

In [38]:
df_meta.show()

+--------------------+--------------------+----------+--------------------+--------------------+----+--------------------+-------+--------------------+---+--------------------+--------------------+------------+--------------------+--------------------+--------------------+-----+-----+--------------------+
|            also_buy|           also_view|      asin|               brand|            category|date|         description|details|             feature|fit|            imageURL|     imageURLHighRes|    main_cat|               price|                rank|        similar_item|tech1|tech2|               title|
+--------------------+--------------------+----------+--------------------+--------------------+----+--------------------+-------+--------------------+---+--------------------+--------------------+------------+--------------------+--------------------+--------------------+-----+-----+--------------------+
|                  []|                  []|0000191639|           Dr. Seuss|[Toy